Esmalt loeb failid listi ning viib nad paremini käideltatavale kujule

In [142]:
from estnltk import Text
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup as bs
import re
import html

name_matcher = r"\<title\>(.+)\<\/title\>"
sample_data = "eesti_seadus"

def is_content_word(word):
    return len(set(word["analysis"][0]["partofspeech"]) & set(['S', 'V', 'A', 'C', 'U', 'D'])) > 0

def lemmafy_text(text):
    xs = Text(text).tag_analysis()
    return " ".join([x["analysis"][0]["lemma"] for x in xs.words if is_content_word(x)])
    

def read_files_from_dir(dir_name):
    total_data = []
    corresponding_names = []

    for f in listdir(dir_name):
        if isfile(join(dir_name, f)):
            content, name = formatted_data(join(dir_name, f))
            total_data.append(lemmafy_text(content))
            if name:
                corresponding_names.append(html.unescape(name.group(1)))
            else:
                corresponding_names.append(f)
    return total_data, corresponding_names

def formatted_data(filename):
    with open(filename, "r") as file:
        content = "".join(file.readlines())
        match = re.search(name_matcher, content)
        bs_content = bs(content, "lxml")
        return bs_content.text, match

corpus, names = read_files_from_dir(sample_data)

In [143]:
# -*- coding: utf-8 -*-
"""
Created on November 4 2019

INFORMATION RETRIAVAL ALGORITHM: TF-IDF variation

@author: Ahti Lohk
"""

from collections import Counter
from math import log

def docs_avg_len(init_doc_dict):
    total = 0
    for value in init_doc_dict.values():
        total += len(value.split())
    return total / len(init_doc_dict.keys())

def len_normalizer(b, doc, avg_doc):
    d = len(doc.split())
    return 1 - b + b * (d / avg_doc)

def term_freq(init_doc_dict):
    term_freq_dict = dict.fromkeys(init_doc_dict.keys())
    for key, value in init_doc_dict.items():
        words_set = set(value.split())
        term_freq_dict[key] = dict()
        for word in words_set:
            word_freq = value.count(word)
            term_freq_dict[key][word] = word_freq
    return term_freq_dict

def query_doc_freq(query):
    words_list = query.split()
    words_set = set(words_list)
    query_doc_freq_dict = dict()
    for word in words_set:
        word_freq = words_list.count(word)
        query_doc_freq_dict[word] = word_freq
    return query_doc_freq_dict

def tf_transformer(term_freq):
    return log(1 + log(1 + term_freq))

def idf(term_freq_dict, query):
    query_words_set = set(query.split())
    idf_dict = dict()
    doc_freq = 0
    nr_of_doc = len(term_freq_dict.keys())
    for word in query_words_set:
        for doc, value in term_freq_dict.items():
            if word in term_freq_dict[doc]:
                doc_freq += 1
                idf_dict[word] = log((nr_of_doc+1) / doc_freq)
    return idf_dict

def docs_scores(init_doc_dict, query_string, b):
    avg_doc_len = docs_avg_len(init_doc_dict)
    term_freq_dict = term_freq(init_doc_dict)
    query_words_freq = query_doc_freq(query_string)
    query_words_set = set(query_string.split())
    idf_dict = idf(term_freq_dict, query_string)
    docs_rank_dict = dict()
    word_scores = Counter()
    for doc, value in term_freq_dict.items():
        total_score = 0
        doc_len = len(doc.split())
        for word in query_words_set:
            if word in term_freq_dict[doc]:
                doc_term_freq = term_freq_dict[doc][word]
            else:
                doc_term_freq = 0
            query_term_freq = query_words_freq[word]
            word_score = query_term_freq * (tf_transformer(doc_term_freq) * idf_dict[word])/ (1 - b + b * (doc_len/avg_doc_len))
            word_scores[word] += word_score
            total_score += word_score
        docs_rank_dict[doc] = round(total_score, 3)
    return docs_rank_dict, word_scores

In [144]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = dict()
for i, data in enumerate(corpus):
    documents[str(i)] = data
    
Tfidf = TfidfVectorizer()
X_Tfidf = Tfidf.fit_transform(corpus)
print("TF-IDF feature names:", len(Tfidf.get_feature_names()))

TF-IDF feature names: 28423


In [145]:
from sklearn.metrics.pairwise import cosine_similarity

idf_results = dict()
idf_var_results = dict()

similarities_tfidf = []
similarities_tfidf_var = []

for i in range(len(names)):
    similarity_tfidf = [-1, -1, -1]
    similarity_tfidf_var = [-1, -1, -1]
    
    docs_scores_dict, word_scores = docs_scores(documents, corpus[i], 0.75)
    
    for j in docs_scores_dict.keys():
        if i != int(j):
            if float(similarity_tfidf_var[0]) > float(docs_scores_dict[j]):
                similarity_tfidf_var = [float(docs_scores_dict[j]), i, int(j)]
    
    idf_var_results[i] = word_scores
    similarities_tfidf_var.append((similarity_tfidf_var[0], similarity_tfidf_var[1], similarity_tfidf_var[2]))
    
    

    for j in range(len(names)):
        if i != j:
            
            tfidf_cosine_sim = cosine_similarity(X_Tfidf[i], X_Tfidf[j])[0][0]
            if tfidf_cosine_sim > similarity_tfidf[0]:
                similarity_tfidf = [tfidf_cosine_sim, i, j]

    idf_results[i] = Counter(idf(term_freq(documents), corpus[i]))
    similarities_tfidf.append((similarity_tfidf[0], similarity_tfidf[1], similarity_tfidf[2]))

In [146]:
from collections import Counter
import pandas as pd
import networkx as nx
from IPython.display import display

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

print("Ülesanne 1\n\n\n")

graphs = [("TF-IDF-var", similarities_tfidf_var), ("TF-IDF", similarities_tfidf)]

for name, graph in graphs:
    popular = Counter()
    formatted_graph = []
    for _, a, b in graph:
        popular[b] += 1
        formatted_graph.append((a, b))

    nx_graph = nx.Graph(formatted_graph)
    largest = set()
    networks = [x for x in nx.connected_components(nx_graph)]
    for network in list(networks):
        if len(network) > len(largest):
            largest = network
        
        
    df = pd.DataFrame([(names[x].split("(")[0].strip(), y) for x, y in popular.most_common(5)], columns = ['Seadus', 'Korduste arv']) 
    print(name + " korduste arvud")
    display(df)
    print("Seotud komponentide arv: " + str(len(networks)))
    print("Suurima komponendi elementide arv: " + str(len(largest)))
    print("Suurima komponendi elemendid: {" + ", ".join([names[x].split("(")[0].strip() for x in largest]) + "}")
    print("\n\n\n")


Ülesanne 1



TF-IDF-var korduste arvud


,Seadus,Korduste arv
0,Tolliseadustik,141
1,Äriseadustik,47
2,Kriminaalmenetluse koodeks,35
3,Väärtpaberituru seadus,23
4,Tsiviilkohtumenetluse seadustik,12


Seotud komponentide arv: 14
Suurima komponendi elementide arv: 176
Suurima komponendi elemendid: {Ravimiseadus, Mõõteseadus, Riigilõivuseadus, Välisteenistuse seadus, Riigireservi seadus, Muinsuskaitseseadus, Isikuandmete kaitse seadus, Rahvatervise seadus, Pühade ja tähtpäevade seadus, Narkootiliste ja psühhotroopsete ainete seadus, Kuvariga töötamise töötervishoiu ja -ohutuse põhinõuded, Riiklike sotsiaaltoetuste maksmise tingimuste ja korra kehtestamine, Arstiabi esimese etapi korraldamine, Tööruumide mikrokliima tervisekaitsenormid ja -eeskirjad TKNE-5/1995, Riikliku statistika seadus, Andmekogude seadus, Mahepõllumajanduse seadus, Tollitariifiseadus, Tolliväärtuse määramise seadus, Planeerimis- ja ehitusseadus, Eriolukorra seadus, Toote nõuetekohasuse tõendamise seadus, Tolliseadus, Valla- ja linnaeelarve ning riigieelarve vahekorra seadus, Kohalike maksude seadus, Põllumajandusloomade tõuaretuse seadus, Väetiseseadus, Maksukorralduse seadus, Tubakaaktsiisi seadus, Kütuseaktsiisi 

,Seadus,Korduste arv
0,Asjaõigusseadus,7
1,Äriseadustik,6
2,Lennundusseadus,6
3,Asjaõigusseaduse rakendamise seadus,5
4,Maareformi seadus,5


Seotud komponentide arv: 100
Suurima komponendi elementide arv: 21
Suurima komponendi elemendid: {Erastamisseadus, Riigi omandisse kuuluvat maavara sisaldavale maatükile ehitise rajamise kord, Asjaõigusseadus, Muinsuskaitseseadus, Maakorraldusseadus, Metsaseadus, Elundite ja kudede siirdamise seadus, Maa hindamise seadus, Maamaksuseadus, Tsiviilseadustiku üldosa seadus, Kommertspandiseadus, Planeerimis- ja ehitusseadus, Asjaõigusseaduse rakendamise seadus, Pärimisseadus, Kinnistusraamatuseadus, Maakatastriseadus, Kinnisasja sundvõõrandamise seadus, Keskkonnamõju hindamise ja keskkonnaauditeerimise seadus, Välismaalasele , välisriigile ... kinnisomandi üleandmise kitsendamise seadus, Riigivaraseadus, Maareformi seadus}






In [148]:
# !pip install gensim
from gensim.models import Word2Vec, KeyedVectors # to load the model
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

model = KeyedVectors.load_word2vec_format('lemmas.sg.s200.w2v.bin', binary=True)

print("Ülesanne 2\n\n\n")

graphs = [("TF-IDF-var", similarities_tfidf_var, idf_var_results), ("TF-IDF", similarities_tfidf, idf_results)]
for name, graph, results in graphs:
    mutated = sorted(graph, key=lambda x: -x[0])
        
    data = []
    global_used = set()
    
    base = results[mutated[0][1]]
    similar = results[mutated[0][2]]
    for term, term_score in base.most_common(10):
        term_similarity = Counter()
        used = set()
        used.add(term)
        for other, _ in similar.items():
            if other not in used:
                try:
                    term_similarity[other] = model.similarity(term, other)
                except Exception:
                    pass
                used.add(other)
        
        if term not in global_used:
            global_used.add(term)
            common = term_similarity.most_common(10)
            data.append([term, ", ".join([x for x, _ in common if x not in global_used])])
            for _, x in common:
                global_used.add(x)
    
    df = pd.DataFrame(data, columns = ['Kõrge ' + name + ' skooriga term', 'Sarnased sõnad'])
    display(df)

Ülesanne 2





,Kõrge TF-IDF-var skooriga term,Sarnased sõnad
0,teisendama,"teisendamine, html-fail, täheline, märgendama, numbriline, alljaotus, järjestamine, allakriipsutus, arvutama, alaindeks"
1,stamp,"mittekommertsiaalne, html-fail, sõnaline, allakriipsutus, teisendamine, šrift, loetud, võõrtäht, eesti-ladin, täheline"
2,väljasõidukohustus,"ühistuseadus, seadus, postisihtnumber, materiaal, tsiviilkohtumenetlus, seadustik, tsiviilseadustik, registriasi, tühistatav, äriregistripidaja"
3,sissesõidukeeld,"vabadusekaotuslik, väljasaatmine, postisihtnumber, aadressiandmed, registriasi, tsiviilkohtumenetlus, äriregistripidaja, karistus, seadustik, materiaal"
4,väljasõit,"elamaasumine, ärajätmine, sõidukulu, trahvimäärus, arvutiregister, tutvumine, päev, majutus, möödumine, lahkumine"
5,siseminister,"justiitsminister, rahandusminister, valitsus, peadirektor, teenistuslik, isiklikult, ametiisik, amet, ülem, likvideerimisabinõu"
6,tähistama,"sünnipäev, nimetama, mööduma, korraldama, tähistus, avama, tähis, tähtpäev, algama, esinema"
7,pitser,"jäljend, pealdis, loetavalt, märk, vorm, pitsat, märge, allkiri, arvutiregister, väljavõte"
8,reisidokument,"registreerimistunnistus, tõendav, dokument, arvutiregister, konsulaarasutus, isikukood, kehtivus, tegevusluba, isikusamasus, registreerimisnumber"
9,hiljemalt,"lisatähtaeg, märkimisaeg, likvideerimisteade, viivitamata, teatis, järgnev, viivitav, kuupäev, sundlõpetamisteade, lõppbilanss"


,Kõrge TF-IDF skooriga term,Sarnased sõnad
0,analoogia,"seos, mõiste, säte, volitatud, meetod, arhiiviametnik, põhjendus, tõestus, lähtuma, paragrahv"
1,tähistama,"nimetama, mööduma, korraldama, algama, märk, toimuma, asutama, möödumine, jäädvustama, alaindeks"
2,reguleerimisala,"säte, kohaldamine, sätestatu, kohaldama, reguleeriv, rakendussäte, üldsäte, õigusakt, sätestama, lõige"
3,küllaldane,"vajalik, nõuetekohane, olemasolu, puudumine, täiendav, asjakohane, tagav, vastav, tagama, säilitustähtaeg"
4,võõrandamine,"võõrandama, omand, võõrandaja, üleandmis-vastuvõtmisakt, ostuõigus, eraarhivaal, omandiõigus, tagastamine, arhiiviregister, müümine"
5,erikonfiskeeritakse,"säilitamistähtaeg, säilitustähtaeg, arhiivieeskiri, arhiiviametnik, volitatud, arhiiviregister, arhiiviinspektor, arhiiviasutus, andmetöötlusalane, arhiiviteenus"
6,haldusõiguserikkumine,"seadustik, halduskaristus, arhiivieeskiri, tõestamisseadus, karistusregister, karistusandmed, arhiiviseadus, paragrahv, seadus, halduskord"
7,keelama,"luba, kohustama, piirama, piirang, keelduma, reguleerima, kehtestama, karistama, loobuma, õigus"
8,arvutuslingvistika,"arhiivindusala, arhiivindusalane, asjaajamisteenus, zcaron, teaduslik, arhiiviinspektor, arhiivindus, mittekommertsiaalne, säilitamisteenus, vallaarhivaar"
9,tervislik,"seisund, säilitustingimus, seksuaalelu, seisukord, soodustav, halb, kõlbmatu, üldine, areng"


Hinnang:

---

Lõppkokkuvõteks
 - Suurim seaduste grupp olid oodatust palju suurem
 - Erinevus TF-IDF ning TF-IDF variatsiooni vahel oli märgatav
 - TF-IDF andis intelligentsemaid tulemusid. TF-IDF variatsioon tekitas kokku 14 erinevat komponentide klastrit, mida on väga vähe arvestades algtekstide arve.
 - Pandas dataframe on igati kasulik
 - TF-IDF variatsioon on hea mõõdik sõna tähtsuse leidmiseks
 - Word2Vec annab häid tulemusid
